# https://plot.ly/python/animations/:

add plotly animations for explanation.

# Useful links for ML and AI for trading

## Forecasting Market Movements Using Tensorflow

#### https://hackernoon.com/forecasting-market-movements-using-tensorflow-fb73e614cd06



## The Rise Of Automated Trading: Machines Trading the S&P 500

#### https://www.toptal.com/machine-learning/s-p-500-automated-trading

## Using LSTMs to forecast time-series

#### https://towardsdatascience.com/using-lstms-to-forecast-time-series-4ab688386b1f

## Time Series Prediction with LSTM Recurrent Neural Networks in Python with Keras

#### https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/

## Decision Tree For Trading Using Python

#### https://www.quantinsti.com/blog/decision-tree

## Machine Learning Logistic Regression In Python: From Theory To Trading

#### https://www.quantinsti.com/blog/machine-learning-logistic-regression-python

## How to Convert a Time Series to a Supervised Learning Problem in Python

#### https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/

## How to Prepare Univariate Time Series Data for Long Short-Term Memory Networks

#### https://machinelearningmastery.com/prepare-univariate-time-series-data-long-short-term-memory-networks/

## Deep Learning in Finance

#### https://towardsdatascience.com/deep-learning-in-finance-9e088cb17c03

# Stock Market Prediction by Recurrent Neural Network on LSTM Model

#### https://medium.com/@aniruddha.choudhury94/stock-market-prediction-by-recurrent-neural-network-on-lstm-model-56de700bff68

##  Jupyter notebook extensions to help increase your productivity.

#### https://www.kdnuggets.com/2018/03/top-5-best-jupyter-notebook-extensions.html

## The Visual Python Debugger for Jupyter Notebooks You’ve Always Wanted
Introducing PixieDebugger

#### https://medium.com/ibm-watson-data-lab/the-visual-python-debugger-for-jupyter-notebooks-youve-always-wanted-761713babc62

## Helpful Functions

In [ ]:
import base64
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from sklearn import preprocessing


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = f"{name}-{tv}"
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m:>02}:{s:>05.2f}"


# Regression chart.
def chart_regression(pred, y, sort=True):
    t = pd.DataFrame({'pred': pred, 'y': y.flatten()})
    if sort:
        t.sort_values(by=['y'], inplace=True)
    plt.plot(t['y'].tolist(), label='expected')
    plt.plot(t['pred'].tolist(), label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean())
                          >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
        * (normalized_high - normalized_low) + normalized_low